In [1]:
!git add Feature_crawler.py
!git add 

On branch master
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   Sample_Crawler.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	Feature_crawler.ipynb
	Tsv_clawler.ipynb
	Tsv_data/
	data/
	poster_test/
	test/

no changes added to commit (use "git add" and/or "git commit -a")


In [77]:
from src.crawler.base_crawler import BaseCrawler
from bs4 import BeautifulSoup
import pandas as pd
from pprint import pprint
import re
import json
import os
from Feature_crawler import Feature_Crawler
from Tsv_crawler import Tsv_Crawler
from sklearn.model_selection import KFold, train_test_split
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor,Grayscale

学習用のデータの取り出し方を指定する

In [87]:
def one_to_three(img):
    if len(img.size)==2:
        transform=Grayscale(num_output_channels=3)
        return transform(img)
    else:
        return img

class push_data(Feature_Crawler):
    def __init__(self,panda):
        self.full=panda
        self.full_title=list(self.full.index)
        self.image_loader=Feature_Crawler(light=True).load_image
        
    def __len__(self):
        return len(self.full)
    
    def __getitem__(self,idx):
        transform = transforms.Compose(
            [
                transforms.Lambda(one_to_three),
                transforms.Resize([300,256]),
                transforms.ToTensor()
            ]
        )
        poster=transform(self.image_loader(self.full_title[idx]))
        #他のデータも
        
        return poster
             

tsvデータと画像等のデータをくっつけて交差検証用にデータ分割する

In [88]:

class integrator():
    def __init__(self,last,n_splits=5,shuffle=True):
        self.fea=Feature_Crawler(last)
        self.tsv=Tsv_Crawler(last)
        self.full=self.tsv.title.join(self.fea.feature,how="outer")
        self.elminate_non_image()
        
        self.splitter = KFold(n_splits=n_splits, shuffle=shuffle)
        self.split_full=[[push_data(self.full.iloc[train]), push_data(self.full.iloc[test])]for train,test in self.splitter.split(self.full)]
        
    def elminate_non_image(self):
        self.full=self.full[self.full['image_exist'] != None]


使用例

In [89]:
train_loader=DataLoader(integrator(10).split_full[0][0],batch_size=5)

poster=next(iter(train_loader))
print(poster.shape)

torch.Size([5, 3, 300, 256])


In [90]:
integrator(10).full

,title,region,language,types,attributes,isOriginalTitle,titleType,primaryTitle,originalTitle,isAdult,...,actor,director,creator,description,datePublished,keywords,aggregateRating,review,duration,trailer
titleId,,,,,,,,,,,,,,,,,,,,,
tt0000001,Карменсіта,UA,\N,imdbDisplay,\N,0,short,Carmencita,Carmencita,0,...,"{'@type': 'Person', 'url': '/name/nm1588970/',...","{'@type': 'Person', 'url': '/name/nm0005690/',...","{'@type': 'Organization', 'url': '/company/co0...",Carmencita is a short starring Carmencita. Per...,1894-03-10,"dance,dancer,music hall,kinetoscope,actress","{'@type': 'AggregateRating', 'ratingCount': 16...","{'@type': 'Review', 'itemReviewed': {'@type': ...",PT1M,NaN
tt0000002,Le clown et ses chiens,\N,\N,original,\N,1,short,Le clown et ses chiens,Le clown et ses chiens,0,...,NaN,"{'@type': 'Person', 'url': '/name/nm0721526/',...",NaN,NaN,1892-10-28,"dog,clown","{'@type': 'AggregateRating', 'ratingCount': 20...",NaN,PT5M,NaN
tt0000003,Sarmanul Pierrot,RO,\N,imdbDisplay,\N,0,short,Pauvre Pierrot,Pauvre Pierrot,0,...,NaN,"{'@type': 'Person', 'url': '/name/nm0721526/',...",NaN,NaN,1892-10-28,early animation,"{'@type': 'AggregateRating', 'ratingCount': 13...","{'@type': 'Review', 'itemReviewed': {'@type': ...",PT4M,NaN
tt0000004,Un bon bock,\N,\N,original,\N,1,short,Un bon bock,Un bon bock,0,...,NaN,"{'@type': 'Person', 'url': '/name/nm0721526/',...",NaN,NaN,1892-10-28,"lost film,three word title,early animation,fir...","{'@type': 'AggregateRating', 'ratingCount': 12...","{'@type': 'Review', 'itemReviewed': {'@type': ...",PT12M,NaN
tt0000005,Blacksmithing Scene,US,\N,alternative,\N,0,short,Blacksmith Scene,Blacksmith Scene,0,...,"[{'@type': 'Person', 'url': '/name/nm0443482/'...","{'@type': 'Person', 'url': '/name/nm0005690/',...","{'@type': 'Organization', 'url': '/company/co0...",Blacksmith Scene is a short starring Charles K...,1893-05-09,"blacksmith,national film registry,beer,two wor...","{'@type': 'AggregateRating', 'ratingCount': 21...","{'@type': 'Review', 'itemReviewed': {'@type': ...",PT1M,NaN
tt0000006,Китайский опиумный притон,RU,\N,imdbDisplay,\N,0,short,Chinese Opium Den,Chinese Opium Den,0,...,NaN,"{'@type': 'Person', 'url': '/name/nm0005690/',...",NaN,NaN,1894-10-17,"lost film,three word title,opium,drugs","{'@type': 'AggregateRating', 'ratingCount': 11...",NaN,PT1M,NaN
tt0000007,Corbett and Courtney Before the Kinetograph,US,\N,\N,\N,0,short,Corbett and Courtney Before the Kinetograph,Corbett and Courtney Before the Kinetograph,0,...,"[{'@type': 'Person', 'url': '/name/nm0179163/'...","[{'@type': 'Person', 'url': '/name/nm0005690/'...","{'@type': 'Organization', 'url': '/company/co0...",Corbett and Courtney Before the Kinetograph is...,1894-11-17,"boxing,bare chested male","{'@type': 'AggregateRating', 'ratingCount': 66...","{'@type': 'Review', 'itemReviewed': {'@type': ...",PT1M,NaN
tt0000008,Edison asszisztense tüsszent,HU,\N,imdbDisplay,\N,0,short,Edison Kinetoscopic Record of a Sneeze,Edison Kinetoscopic Record of a Sneeze,0,...,"{'@type': 'Person', 'url': '/name/nm0653028/',...","{'@type': 'Person', 'url': '/name/nm0005690/',...","{'@type': 'Organization', 'url': '/company/co0...",Edison Kinetoscopic Record of a Sneeze is a sh...,1894-01-09,"national film registry,sneeze,snuff powder,sne...","{'@type': 'AggregateRating', 'ratingCount': 18...","{'@type': 'Review', 'itemReviewed': {'@type': ...",PT1M,NaN
tt0000009,Miss Jerry,\N,\N,original,\N,1,movie,Miss Jerry,Miss Jerry,0,...,"[{'@type': 'Person', 'url': '/name/nm0063086/'...","{'@type': 'Person', 'url': '/name/nm0085156/',...","[{'@type': 'Person', 'url': '/name/nm0085156/'...",Miss Jerry is a movie starring Blanche Bayliss...,1894-10-09,"character name as title,two word title,nicknam...","{'@type': 'AggregateRating', 'ratingCount': 15...","{'@type': 'Review', 'itemReviewed': {'@type': ...",PT45M,NaN
